In [ ]:
import pandas as pd 
import numpy as np 

def get_pareto_area (df ,target_cols ):
    """
    （），。
     Series，。
    """

    data =df [target_cols ].values 

    area =np .prod (data +1e-6 ,axis =1 )
    return pd .Series (area ,index =df .index )

import numpy as np 
import pandas as pd 

def get_radar_area (df ,target_cols ):
    """
    （），。
     get_pareto_area 。
    """
    n =len (target_cols )
    theta =2 *np .pi /n 
    areas =[]

    for _ ,row in df [target_cols ].iterrows ():
        values =row .values .astype (float )
        values =np .log1p (values )
        area =0.0 
        for i in range (n ):
            a =values [i ]
            b =values [(i +1 )%n ]
            area +=a *b *np .sin (theta )
        area *=0.5 
        areas .append (area )

    return pd .Series (areas ,index =df .index )





file_path ='/data/home/wxl22/changzu_ans/dataset/processed/97RA.xlsx'
df =pd .read_excel (file_path )


target_cols =['Hinesol','beta-Eudesmol','Agarospirol','Atractylon','Elemol','Naphtho[2,3-b]furan, 4,4a,8a,9-tetrahydro-3,5,8a-trimethyl-','Atractylodin','gamma-Elemene','Longipinocarvone','Hedycaryol']



pareto_mask =get_radar_area (df ,target_cols )


df ['pareto_value']=pareto_mask 


df .to_excel ('/data/home/wxl22/changzu_ans/dataset/processed/pareto_result.xlsx',index =False )

print ("， pareto_result.xlsx")


， pareto_result.xlsx


In [ ]:
import numpy as np 
import matplotlib .pyplot as plt 
from matplotlib .colors import ListedColormap ,BoundaryNorm 
import geopandas as gpd 
from shapely .geometry import Point 
from sklearn .neighbors import KDTree 


try :
    from pykrige .ok import OrdinaryKriging 
    _HAS_PYKRIGE =True 
except ImportError :
    _HAS_PYKRIGE =False 


def draw_demo_map (method ="idw",show_uncertainty =False ):

    np .random .seed (42 )
    lon =np .random .uniform (73 ,135 ,150 )
    lat =np .random .uniform (18 ,54 ,150 )
    vals =np .random .uniform (1000 ,2500 ,150 )
    bins =[-np .inf ,1200 ,1600 ,2000 ,2400 ,np .inf ]


    world =gpd .read_file (gpd .datasets .get_path ("naturalearth_lowres"))
    prov =world [world .name =="China"]
    boundary =prov .dissolve ()
    china_poly =boundary .geometry .unary_union 


    prov_m =prov .to_crs (3857 )
    boundary_m =prov_m .dissolve ()
    china_poly_m =boundary_m .geometry .unary_union 

    gpt =gpd .GeoDataFrame ({"v":vals },geometry =gpd .points_from_xy (lon ,lat ),crs =4326 ).to_crs (3857 )


    x0 ,y0 ,x1 ,y1 =prov_m .total_bounds 
    nx ,ny =200 ,150 
    xs =np .linspace (x0 ,x1 ,nx )
    ys =np .linspace (y0 ,y1 ,ny )
    xx_m ,yy_m =np .meshgrid (xs ,ys )

    if method =="idw":
        xy =np .c_ [gpt .geometry .x ,gpt .geometry .y ]
        tree =KDTree (xy )
        dist ,idx =tree .query (np .c_ [xx_m .ravel (),yy_m .ravel ()],k =8 )
        w =1.0 /np .maximum (dist ,1e-6 )**2 
        zi =np .sum (w *vals [idx ],axis =1 )/np .sum (w ,axis =1 )
        zi =zi .reshape (xx_m .shape )
        ss =None 
    elif method =="kriging"and _HAS_PYKRIGE :
        OK =OrdinaryKriging (gpt .geometry .x ,gpt .geometry .y ,vals ,
        variogram_model ="spherical",
        verbose =False ,enable_plotting =False )
        zi ,ss =OK .execute ("grid",xs ,ys )
        zi =np .array (zi )
        ss =np .array (ss )
    else :
        return None 


    pts_m =gpd .GeoSeries ([Point (x ,y )for x ,y in zip (xx_m .ravel (),yy_m .ravel ())],crs =3857 )
    mask =pts_m .within (china_poly_m ).to_numpy ().reshape (xx_m .shape )
    zi [~mask ]=np .nan 
    if method =="kriging"and ss is not None :
        ss [~mask ]=np .nan 


    grid_ll =gpd .GeoDataFrame (geometry =pts_m ,crs =3857 ).to_crs (4326 )
    xx =grid_ll .geometry .x .to_numpy ().reshape (xx_m .shape )
    yy =grid_ll .geometry .y .to_numpy ().reshape (yy_m .shape )


    cmap_colors =["#D9B300","#5AA1A4","#C56B5B","#3F6BAA","#6FAE5B"]
    cmap =ListedColormap (cmap_colors ,name ="zones")
    norm =BoundaryNorm (bins ,ncolors =cmap .N )

    fig ,ax =plt .subplots (figsize =(8 ,6 ),dpi =150 )
    pcm =ax .pcolormesh (xx ,yy ,zi ,cmap =cmap ,norm =norm ,shading ="auto")
    prov .boundary .plot (ax =ax ,color ="black",linewidth =0.8 )
    ax .set_xlim (73 ,135 )
    ax .set_ylim (18 ,54 )
    ax .set_title (f"Suitability zones demo ({method})")

    plt .colorbar (pcm ,ax =ax )
    plt .show ()


    if show_uncertainty and method =="kriging"and ss is not None :
        fig2 ,ax2 =plt .subplots (figsize =(8 ,6 ),dpi =150 )
        im =ax2 .pcolormesh (xx ,yy ,ss ,cmap ="magma",shading ="auto")
        prov .boundary .plot (ax =ax2 ,color ="black",linewidth =0.8 )
        ax2 .set_xlim (73 ,135 )
        ax2 .set_ylim (18 ,54 )
        ax2 .set_title ("Kriging variance demo")
        plt .colorbar (im ,ax =ax2 )
        plt .show ()



draw_demo_map ("idw")
if _HAS_PYKRIGE :
    draw_demo_map ("kriging",show_uncertainty =True )
